# 模型训练和评估验证

# 导入包

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import time
import re

# 导入特征工程后的数据

In [3]:
inputfile = '../datasets/data_feature_processed.csv'
modelfile = '../model/xgb.pkl'

In [26]:
data = pd.read_csv(inputfile)

# XGBoost模型训练

In [37]:
df = data.copy()
df.sort_values(by=['loc_id','time_stamp'],inplace=True)

In [38]:
df.head()

,time_stamp,loc_id,num_of_people,hour_of_day,month_of_year,day_of_week,week_of_year,history_1h_population,history_2h_population,history_3h_population,...,24,25,26,27,28,29,30,31,32,33
0,2017-04-01 00:00:00,1,172.0,0,4,5,13,208.0,329.0,368.0,...,0,0,0,0,0,0,0,0,0,0
1,2017-04-01 01:00:00,1,192.0,1,4,5,13,172.0,208.0,329.0,...,0,0,0,0,0,0,0,0,0,0
2,2017-04-01 02:00:00,1,190.0,2,4,5,13,192.0,172.0,208.0,...,0,0,0,0,0,0,0,0,0,0
3,2017-04-01 03:00:00,1,177.0,3,4,5,13,190.0,192.0,172.0,...,0,0,0,0,0,0,0,0,0,0
4,2017-04-01 04:00:00,1,216.0,4,4,5,13,177.0,190.0,192.0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.tail()

,time_stamp,loc_id,num_of_people,hour_of_day,month_of_year,day_of_week,week_of_year,history_1h_population,history_2h_population,history_3h_population,...,24,25,26,27,28,29,30,31,32,33
163147,2017-10-31 19:00:00,33,407.0,19,10,1,44,919.0,1540.0,740.0,...,0,0,0,0,0,0,0,0,0,1
163148,2017-10-31 20:00:00,33,496.0,20,10,1,44,407.0,919.0,1540.0,...,0,0,0,0,0,0,0,0,0,1
163149,2017-10-31 21:00:00,33,434.0,21,10,1,44,496.0,407.0,919.0,...,0,0,0,0,0,0,0,0,0,1
163150,2017-10-31 22:00:00,33,272.0,22,10,1,44,434.0,496.0,407.0,...,0,0,0,0,0,0,0,0,0,1
163151,2017-10-31 23:00:00,33,120.0,23,10,1,44,272.0,434.0,496.0,...,0,0,0,0,0,0,0,0,0,1


## 1. 划分训练集和测试集

In [40]:
train = df
test = df[df.time_stamp>='2017-10-01']
train_X = train.iloc[:,3:]
train_y = train['num_of_people']
test_X = test.iloc[:,3:]
test_y = test['num_of_people']
print(train_X.shape,test_X.shape)

(163152, 90) (18216, 90)


## 2. XGBoost训练

In [41]:
model_xgb = xgboost_model_train(train_X,train_y,test_X,test_y)

[0]	train-rmse:693.909	test-rmse:1039.55
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:674.878	test-rmse:1011.63
[2]	train-rmse:656.41	test-rmse:983.985
[3]	train-rmse:638.788	test-rmse:957.711
[4]	train-rmse:621.439	test-rmse:931.868
[5]	train-rmse:604.633	test-rmse:907.27
[6]	train-rmse:588.308	test-rmse:882.817
[7]	train-rmse:572.804	test-rmse:859.505
[8]	train-rmse:557.461	test-rmse:836.386
[9]	train-rmse:542.599	test-rmse:814.368
[10]	train-rmse:528.146	test-rmse:792.681
[11]	train-rmse:514.218	test-rmse:771.902
[12]	train-rmse:500.766	test-rmse:751.8
[13]	train-rmse:487.696	test-rmse:732.445
[14]	train-rmse:475.031	test-rmse:713.603
[15]	train-rmse:462.721	test-rmse:694.976
[16]	train-rmse:450.858	test-rmse:677.147
[17]	train-rmse:439.592	test-rmse:660.234
[18]	train-rmse:428.481	test-rmse:643.654
[19]	train-rmse:417.678	test-rmse:627.356
[20]	train-rmse:407.422	test-rm

[191]	train-rmse:123.782	test-rmse:168.39
[192]	train-rmse:123.721	test-rmse:168.256
[193]	train-rmse:123.638	test-rmse:168.112
[194]	train-rmse:123.502	test-rmse:167.942
[195]	train-rmse:123.421	test-rmse:167.81
[196]	train-rmse:123.385	test-rmse:167.66
[197]	train-rmse:123.324	test-rmse:167.588
[198]	train-rmse:123.226	test-rmse:167.348
[199]	train-rmse:123.157	test-rmse:167.237
[200]	train-rmse:123.063	test-rmse:167.019
[201]	train-rmse:123.006	test-rmse:166.901
[202]	train-rmse:122.888	test-rmse:166.775
[203]	train-rmse:122.791	test-rmse:166.672
[204]	train-rmse:122.747	test-rmse:166.576
[205]	train-rmse:122.672	test-rmse:166.499
[206]	train-rmse:122.617	test-rmse:166.367
[207]	train-rmse:122.549	test-rmse:166.256
[208]	train-rmse:122.481	test-rmse:166.097
[209]	train-rmse:122.445	test-rmse:166.004
[210]	train-rmse:122.331	test-rmse:165.847
[211]	train-rmse:122.232	test-rmse:165.74
[212]	train-rmse:122.206	test-rmse:165.676
[213]	train-rmse:122.173	test-rmse:165.578
[214]	train-rms

[383]	train-rmse:113.268	test-rmse:151.66
[384]	train-rmse:113.224	test-rmse:151.627
[385]	train-rmse:113.206	test-rmse:151.573
[386]	train-rmse:113.168	test-rmse:151.497
[387]	train-rmse:113.128	test-rmse:151.409
[388]	train-rmse:113.096	test-rmse:151.342
[389]	train-rmse:113.058	test-rmse:151.179
[390]	train-rmse:113.019	test-rmse:151.15
[391]	train-rmse:112.98	test-rmse:151.1
[392]	train-rmse:112.924	test-rmse:151.071
[393]	train-rmse:112.87	test-rmse:150.917
[394]	train-rmse:112.839	test-rmse:150.846
[395]	train-rmse:112.818	test-rmse:150.795
[396]	train-rmse:112.798	test-rmse:150.733
[397]	train-rmse:112.753	test-rmse:150.679
[398]	train-rmse:112.736	test-rmse:150.644
[399]	train-rmse:112.722	test-rmse:150.593
[400]	train-rmse:112.698	test-rmse:150.548
[401]	train-rmse:112.68	test-rmse:150.493
[402]	train-rmse:112.657	test-rmse:150.461
[403]	train-rmse:112.629	test-rmse:150.42
[404]	train-rmse:112.587	test-rmse:150.348
[405]	train-rmse:112.539	test-rmse:150.269
[406]	train-rmse:11

[575]	train-rmse:107.463	test-rmse:142.598
[576]	train-rmse:107.421	test-rmse:142.543
[577]	train-rmse:107.405	test-rmse:142.501
[578]	train-rmse:107.374	test-rmse:142.473
[579]	train-rmse:107.336	test-rmse:142.415
[580]	train-rmse:107.312	test-rmse:142.38
[581]	train-rmse:107.292	test-rmse:142.345
[582]	train-rmse:107.258	test-rmse:142.308
[583]	train-rmse:107.231	test-rmse:142.266
[584]	train-rmse:107.202	test-rmse:142.245
[585]	train-rmse:107.194	test-rmse:142.212
[586]	train-rmse:107.165	test-rmse:142.18
[587]	train-rmse:107.134	test-rmse:142.149
[588]	train-rmse:107.108	test-rmse:142.107
[589]	train-rmse:107.1	test-rmse:142.08
[590]	train-rmse:107.081	test-rmse:142.05
[591]	train-rmse:107.058	test-rmse:142.008
[592]	train-rmse:107.013	test-rmse:141.956
[593]	train-rmse:106.978	test-rmse:141.911
[594]	train-rmse:106.947	test-rmse:141.871
[595]	train-rmse:106.916	test-rmse:141.839
[596]	train-rmse:106.881	test-rmse:141.796
[597]	train-rmse:106.857	test-rmse:141.769
[598]	train-rmse:

[767]	train-rmse:103.129	test-rmse:136.069
[768]	train-rmse:103.122	test-rmse:136.043
[769]	train-rmse:103.103	test-rmse:136.019
[770]	train-rmse:103.084	test-rmse:135.982
[771]	train-rmse:103.075	test-rmse:135.951
[772]	train-rmse:103.053	test-rmse:135.933
[773]	train-rmse:103.027	test-rmse:135.896
[774]	train-rmse:103.006	test-rmse:135.873
[775]	train-rmse:103.001	test-rmse:135.851
[776]	train-rmse:102.979	test-rmse:135.816
[777]	train-rmse:102.97	test-rmse:135.787
[778]	train-rmse:102.963	test-rmse:135.767
[779]	train-rmse:102.931	test-rmse:135.727
[780]	train-rmse:102.905	test-rmse:135.686
[781]	train-rmse:102.873	test-rmse:135.64
[782]	train-rmse:102.846	test-rmse:135.618
[783]	train-rmse:102.83	test-rmse:135.597
[784]	train-rmse:102.811	test-rmse:135.562
[785]	train-rmse:102.8	test-rmse:135.545
[786]	train-rmse:102.782	test-rmse:135.51
[787]	train-rmse:102.751	test-rmse:135.495
[788]	train-rmse:102.741	test-rmse:135.482
[789]	train-rmse:102.724	test-rmse:135.469
[790]	train-rmse:

[959]	train-rmse:99.7617	test-rmse:131.194
[960]	train-rmse:99.7493	test-rmse:131.174
[961]	train-rmse:99.7409	test-rmse:131.156
[962]	train-rmse:99.7265	test-rmse:131.128
[963]	train-rmse:99.7088	test-rmse:131.114
[964]	train-rmse:99.6885	test-rmse:131.092
[965]	train-rmse:99.674	test-rmse:131.088
[966]	train-rmse:99.654	test-rmse:131.075
[967]	train-rmse:99.6415	test-rmse:131.059
[968]	train-rmse:99.6219	test-rmse:131.028
[969]	train-rmse:99.6004	test-rmse:131.008
[970]	train-rmse:99.5838	test-rmse:130.98
[971]	train-rmse:99.5695	test-rmse:130.968
[972]	train-rmse:99.5349	test-rmse:130.935
[973]	train-rmse:99.5224	test-rmse:130.912
[974]	train-rmse:99.5017	test-rmse:130.887
[975]	train-rmse:99.4769	test-rmse:130.858
[976]	train-rmse:99.4638	test-rmse:130.85
[977]	train-rmse:99.4446	test-rmse:130.836
[978]	train-rmse:99.4163	test-rmse:130.803
[979]	train-rmse:99.4003	test-rmse:130.798
[980]	train-rmse:99.3897	test-rmse:130.777
[981]	train-rmse:99.3806	test-rmse:130.753
[982]	train-rms

## 3. 11月份预测导出

In [42]:
nov_data = pd.read_csv('../datasets/Nov_test_data.csv')

In [43]:
%time nov_data = real_time_predict(model_xgb,nov_data)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[ 419.]
第0行预测完成！
[ 359.]
第1行预测完成！
[ 327.]
第2行预测完成！
[ 296.]
第3行预测完成！
[ 303.]
第4行预测完成！
[ 313.]
第5行预测完成！
[ 290.]
第6行预测完成！
[ 513.]
第7行预测完成！
[ 693.]
第8行预测完成！
[ 654.]
第9行预测完成！
[ 535.]
第10行预测完成！
[ 755.]
第11行预测完成！
[ 1068.]
第12行预测完成！
[ 829.]
第13行预测完成！
[ 598.]
第14行预测完成！
[ 547.]
第15行预测完成！
[ 548.]
第16行预测完成！
[ 899.]
第17行预测完成！
[ 829.]
第18行预测完成！
[ 599.]
第19行预测完成！
[ 558.]
第20行预测完成！
[ 610.]
第21行预测完成！
[ 639.]
第22行预测完成！
[ 415.]
第23行预测完成！
[ 360.]
第24行预测完成！
[ 317.]
第25行预测完成！
[ 274.]
第26行预测完成！
[ 274.]
第27行预测完成！
[ 262.]
第28行预测完成！
[ 293.]
第29行预测完成！
[ 293.]
第30行预测完成！
[ 536.]
第31行预测完成！
[ 684.]
第32行预测完成！
[ 630.]
第33行预测完成！
[ 550.]
第34行预测完成！
[ 810.]
第35行预测完成！
[ 1053.]
第36行预测完成！
[ 835.]
第37行预测完成！
[ 610.]
第38行预测完成！
[ 565.]
第39行预测完成！
[ 539.]
第40行预测完成！
[ 892.]
第41行预测完成！
[ 842.]
第42行预测完成！
[ 635.]
第43行预测完成！
[ 513.]
第44行预测完成！
[ 574.]
第45行预测完成！
[ 585.]
第46行预测完成！
[ 387.]
第47行预测完成！
[ 292.]
第48行预测完成！
[ 338.]
第49行预测完成！
[ 288.]
第50行预测完成！
[ 283.]
第51行预测完成！
[ 273.]
第52行预测完成！
[ 279.]
第53行预测完成！
[ 286.]
第54行预测完成！
[ 468.]
第55行预测完成！


第441行预测完成！
[ 388.]
第442行预测完成！
[ 653.]
第443行预测完成！
[ 958.]
第444行预测完成！
[ 665.]
第445行预测完成！
[ 509.]
第446行预测完成！
[ 425.]
第447行预测完成！
[ 405.]
第448行预测完成！
[ 732.]
第449行预测完成！
[ 755.]
第450行预测完成！
[ 552.]
第451行预测完成！
[ 407.]
第452行预测完成！
[ 394.]
第453行预测完成！
[ 390.]
第454行预测完成！
[ 262.]
第455行预测完成！
[ 221.]
第456行预测完成！
[ 201.]
第457行预测完成！
[ 202.]
第458行预测完成！
[ 224.]
第459行预测完成！
[ 223.]
第460行预测完成！
[ 247.]
第461行预测完成！
[ 300.]
第462行预测完成！
[ 493.]
第463行预测完成！
[ 623.]
第464行预测完成！
[ 560.]
第465行预测完成！
[ 463.]
第466行预测完成！
[ 779.]
第467行预测完成！
[ 1117.]
第468行预测完成！
[ 840.]
第469行预测完成！
[ 615.]
第470行预测完成！
[ 521.]
第471行预测完成！
[ 484.]
第472行预测完成！
[ 897.]
第473行预测完成！
[ 894.]
第474行预测完成！
[ 632.]
第475行预测完成！
[ 511.]
第476行预测完成！
[ 513.]
第477行预测完成！
[ 536.]
第478行预测完成！
[ 348.]
第479行预测完成！
[ 262.]
第480行预测完成！
[ 222.]
第481行预测完成！
[ 211.]
第482行预测完成！
[ 221.]
第483行预测完成！
[ 227.]
第484行预测完成！
[ 246.]
第485行预测完成！
[ 290.]
第486行预测完成！
[ 565.]
第487行预测完成！
[ 710.]
第488行预测完成！
[ 619.]
第489行预测完成！
[ 498.]
第490行预测完成！
[ 824.]
第491行预测完成！
[ 1172.]
第492行预测完成！
[ 878.]
第493行预测完成！

第158行预测完成！
[ 643.]
第159行预测完成！
[ 510.]
第160行预测完成！
[ 955.]
第161行预测完成！
[ 876.]
第162行预测完成！
[ 609.]
第163行预测完成！
[ 530.]
第164行预测完成！
[ 554.]
第165行预测完成！
[ 673.]
第166行预测完成！
[ 574.]
第167行预测完成！
[ 591.]
第168行预测完成！
[ 685.]
第169行预测完成！
[ 743.]
第170行预测完成！
[ 835.]
第171行预测完成！
[ 951.]
第172行预测完成！
[ 1025.]
第173行预测完成！
[ 935.]
第174行预测完成！
[ 895.]
第175行预测完成！
[ 750.]
第176行预测完成！
[ 586.]
第177行预测完成！
[ 458.]
第178行预测完成！
[ 740.]
第179行预测完成！
[ 1121.]
第180行预测完成！
[ 913.]
第181行预测完成！
[ 647.]
第182行预测完成！
[ 595.]
第183行预测完成！
[ 489.]
第184行预测完成！
[ 908.]
第185行预测完成！
[ 859.]
第186行预测完成！
[ 598.]
第187行预测完成！
[ 496.]
第188行预测完成！
[ 536.]
第189行预测完成！
[ 638.]
第190行预测完成！
[ 536.]
第191行预测完成！
[ 530.]
第192行预测完成！
[ 670.]
第193行预测完成！
[ 727.]
第194行预测完成！
[ 834.]
第195行预测完成！
[ 970.]
第196行预测完成！
[ 996.]
第197行预测完成！
[ 922.]
第198行预测完成！
[ 864.]
第199行预测完成！
[ 732.]
第200行预测完成！
[ 583.]
第201行预测完成！
[ 469.]
第202行预测完成！
[ 732.]
第203行预测完成！
[ 1095.]
第204行预测完成！
[ 908.]
第205行预测完成！
[ 662.]
第206行预测完成！
[ 597.]
第207行预测完成！
[ 492.]
第208行预测完成！
[ 936.]
第209行预测完成！
[ 876.]
第210行预测完成

第592行预测完成！
[ 710.]
第593行预测完成！
[ 720.]
第594行预测完成！
[ 510.]
第595行预测完成！
[ 410.]
第596行预测完成！
[ 406.]
第597行预测完成！
[ 479.]
第598行预测完成！
[ 386.]
第599行预测完成！
[ 389.]
第600行预测完成！
[ 442.]
第601行预测完成！
[ 554.]
第602行预测完成！
[ 665.]
第603行预测完成！
[ 761.]
第604行预测完成！
[ 938.]
第605行预测完成！
[ 988.]
第606行预测完成！
[ 982.]
第607行预测完成！
[ 932.]
第608行预测完成！
[ 772.]
第609行预测完成！
[ 547.]
第610行预测完成！
[ 812.]
第611行预测完成！
[ 1057.]
第612行预测完成！
[ 896.]
第613行预测完成！
[ 678.]
第614行预测完成！
[ 555.]
第615行预测完成！
[ 454.]
第616行预测完成！
[ 738.]
第617行预测完成！
[ 749.]
第618行预测完成！
[ 536.]
第619行预测完成！
[ 404.]
第620行预测完成！
[ 391.]
第621行预测完成！
[ 448.]
第622行预测完成！
[ 390.]
第623行预测完成！
[ 454.]
第624行预测完成！
[ 547.]
第625行预测完成！
[ 688.]
第626行预测完成！
[ 786.]
第627行预测完成！
[ 897.]
第628行预测完成！
[ 1090.]
第629行预测完成！
[ 1160.]
第630行预测完成！
[ 1198.]
第631行预测完成！
[ 1157.]
第632行预测完成！
[ 918.]
第633行预测完成！
[ 663.]
第634行预测完成！
[ 939.]
第635行预测完成！
[ 1207.]
第636行预测完成！
[ 1035.]
第637行预测完成！
[ 800.]
第638行预测完成！
[ 644.]
第639行预测完成！
[ 536.]
第640行预测完成！
[ 909.]
第641行预测完成！
[ 876.]
第642行预测完成！
[ 610.]
第643行预测完成！
[ 496.]
第644行

...

第259行预测完成！
[ 939.]
第260行预测完成！
[ 881.]
第261行预测完成！
[ 893.]
第262行预测完成！
[ 332.]
第263行预测完成！
[ 69.]
第264行预测完成！
[ 16.]
第265行预测完成！
[ 2.]
第266行预测完成！
[ 2.]
第267行预测完成！
[-1.]
第268行预测完成！
[ 12.]
第269行预测完成！
[ 91.]
第270行预测完成！
[ 360.]
第271行预测完成！
[ 795.]
第272行预测完成！
[ 966.]
第273行预测完成！
[ 825.]
第274行预测完成！
[ 1452.]
第275行预测完成！
[ 1956.]
第276行预测完成！
[ 1293.]
第277行预测完成！
[ 937.]
第278行预测完成！
[ 944.]
第279行预测完成！
[ 955.]
第280行预测完成！
[ 1627.]
第281行预测完成！
[ 1667.]
第282行预测完成！
[ 1252.]
第283行预测完成！
[ 922.]
第284行预测完成！
[ 872.]
第285行预测完成！
[ 872.]
第286行预测完成！
[ 328.]
第287行预测完成！
[ 78.]
第288行预测完成！
[ 34.]
第289行预测完成！
[ 19.]
第290行预测完成！
[ 13.]
第291行预测完成！
[ 10.]
第292行预测完成！
[ 20.]
第293行预测完成！
[ 111.]
第294行预测完成！
[ 649.]
第295行预测完成！
[ 1232.]
第296行预测完成！
[ 1119.]
第297行预测完成！
[ 892.]
第298行预测完成！
[ 1880.]
第299行预测完成！
[ 2544.]
第300行预测完成！
[ 1545.]
第301行预测完成！
[ 1168.]
第302行预测完成！
[ 1098.]
第303行预测完成！
[ 1074.]
第304行预测完成！
[ 2166.]
第305行预测完成！
[ 2058.]
第306行预测完成！
[ 1408.]
第307行预测完成！
[ 1116.]
第308行预测完成！
[ 1109.]
第309行预测完成！
[ 1150.]
第310行预测完成！
[ 447.]
第311行预测完

第690行预测完成！
[ 1359.]
第691行预测完成！
[ 1087.]
第692行预测完成！
[ 1018.]
第693行预测完成！
[ 997.]
第694行预测完成！
[ 318.]
第695行预测完成！
[ 9.]
第696行预测完成！
[ 4.]
第697行预测完成！
[ 6.]
第698行预测完成！
[ 8.]
第699行预测完成！
[ 12.]
第700行预测完成！
[ 20.]
第701行预测完成！
[ 135.]
第702行预测完成！
[ 812.]
第703行预测完成！
[ 1497.]
第704行预测完成！
[ 1163.]
第705行预测完成！
[ 964.]
第706行预测完成！
[ 1945.]
第707行预测完成！
[ 2828.]
第708行预测完成！
[ 1614.]
第709行预测完成！
[ 1200.]
第710行预测完成！
[ 1075.]
第711行预测完成！
[ 1096.]
第712行预测完成！
[ 2100.]
第713行预测完成！
[ 1893.]
第714行预测完成！
[ 1277.]
第715行预测完成！
[ 1006.]
第716行预测完成！
[ 927.]
第717行预测完成！
[ 900.]
第718行预测完成！
[ 297.]
第719行预测完成！
******删减前地点数据大小：1440******
******删减后地点数据大小：720******
[ 80.]
第0行预测完成！
[ 39.]
第1行预测完成！
[ 24.]
第2行预测完成！
[ 25.]
第3行预测完成！
[ 27.]
第4行预测完成！
[ 29.]
第5行预测完成！
[ 84.]
第6行预测完成！
[ 310.]
第7行预测完成！
[ 589.]
第8行预测完成！
[ 428.]
第9行预测完成！
[ 322.]
第10行预测完成！
[ 630.]
第11行预测完成！
[ 981.]
第12行预测完成！
[ 610.]
第13行预测完成！
[ 467.]
第14行预测完成！
[ 386.]
第15行预测完成！
[ 361.]
第16行预测完成！
[ 751.]
第17行预测完成！
[ 794.]
第18行预测完成！
[ 548.]
第19行预测完成！
[ 421.]
第20行预测完成！
[ 414.]
第21行预测完成！
[

第415行预测完成！
[ 263.]
第416行预测完成！
[ 329.]
第417行预测完成！
[ 292.]
第418行预测完成！
[ 493.]
第419行预测完成！
[ 741.]
第420行预测完成！
[ 451.]
第421行预测完成！
[ 339.]
第422行预测完成！
[ 298.]
第423行预测完成！
[ 322.]
第424行预测完成！
[ 537.]
第425行预测完成！
[ 608.]
第426行预测完成！
[ 427.]
第427行预测完成！
[ 341.]
第428行预测完成！
[ 328.]
第429行预测完成！
[ 317.]
第430行预测完成！
[ 135.]
第431行预测完成！
[ 42.]
第432行预测完成！
[ 5.]
第433行预测完成！
[-5.]
第434行预测完成！
[-6.]
第435行预测完成！
[-3.]
第436行预测完成！
[ 1.]
第437行预测完成！
[ 18.]
第438行预测完成！
[ 100.]
第439行预测完成！
[ 241.]
第440行预测完成！
[ 320.]
第441行预测完成！
[ 281.]
第442行预测完成！
[ 487.]
第443行预测完成！
[ 713.]
第444行预测完成！
[ 477.]
第445行预测完成！
[ 342.]
第446行预测完成！
[ 291.]
第447行预测完成！
[ 306.]
第448行预测完成！
[ 547.]
第449行预测完成！
[ 626.]
第450行预测完成！
[ 432.]
第451行预测完成！
[ 318.]
第452行预测完成！
[ 292.]
第453行预测完成！
[ 284.]
第454行预测完成！
[ 124.]
第455行预测完成！
[ 40.]
第456行预测完成！
[ 14.]
第457行预测完成！
[ 9.]
第458行预测完成！
[ 7.]
第459行预测完成！
[ 9.]
第460行预测完成！
[ 18.]
第461行预测完成！
[ 40.]
第462行预测完成！
[ 166.]
第463行预测完成！
[ 424.]
第464行预测完成！
[ 386.]
第465行预测完成！
[ 306.]
第466行预测完成！
[ 570.]
第467行预测完成！
[ 869.]
第468行预测完成！
[ 56

第141行预测完成！
[ 295.]
第142行预测完成！
[ 89.]
第143行预测完成！
[ 15.]
第144行预测完成！
[ 2.]
第145行预测完成！
[ 2.]
第146行预测完成！
[ 5.]
第147行预测完成！
[ 8.]
第148行预测完成！
[ 19.]
第149行预测完成！
[ 53.]
第150行预测完成！
[ 270.]
第151行预测完成！
[ 543.]
第152行预测完成！
[ 748.]
第153行预测完成！
[ 621.]
第154行预测完成！
[ 802.]
第155行预测完成！
[ 722.]
第156行预测完成！
[ 612.]
第157行预测完成！
[ 546.]
第158行预测完成！
[ 773.]
第159行预测完成！
[ 631.]
第160行预测完成！
[ 1053.]
第161行预测完成！
[ 735.]
第162行预测完成！
[ 467.]
第163行预测完成！
[ 415.]
第164行预测完成！
[ 346.]
第165行预测完成！
[ 256.]
第166行预测完成！
[ 62.]
第167行预测完成！
[ 3.]
第168行预测完成！
[-5.]
第169行预测完成！
[ 3.]
第170行预测完成！
[ 4.]
第171行预测完成！
[ 6.]
第172行预测完成！
[ 22.]
第173行预测完成！
[ 71.]
第174行预测完成！
[ 290.]
第175行预测完成！
[ 565.]
第176行预测完成！
[ 709.]
第177行预测完成！
[ 628.]
第178行预测完成！
[ 779.]
第179行预测完成！
[ 715.]
第180行预测完成！
[ 588.]
第181行预测完成！
[ 570.]
第182行预测完成！
[ 726.]
第183行预测完成！
[ 628.]
第184行预测完成！
[ 933.]
第185行预测完成！
[ 680.]
第186行预测完成！
[ 447.]
第187行预测完成！
[ 372.]
第188行预测完成！
[ 314.]
第189行预测完成！
[ 220.]
第190行预测完成！
[ 57.]
第191行预测完成！
[ 10.]
第192行预测完成！
[-1.]
第193行预测完成！
[ 1.]
第194行预测完成！
[ 4.]
第195行预

第591行预测完成！
[ 336.]
第592行预测完成！
[ 436.]
第593行预测完成！
[ 433.]
第594行预测完成！
[ 333.]
第595行预测完成！
[ 274.]
第596行预测完成！
[ 222.]
第597行预测完成！
[ 158.]
第598行预测完成！
[ 49.]
第599行预测完成！
[ 5.]
第600行预测完成！
[-7.]
第601行预测完成！
[-7.]
第602行预测完成！
[-5.]
第603行预测完成！
[-3.]
第604行预测完成！
[ 2.]
第605行预测完成！
[ 14.]
第606行预测完成！
[ 95.]
第607行预测完成！
[ 218.]
第608行预测完成！
[ 372.]
第609行预测完成！
[ 381.]
第610行预测完成！
[ 445.]
第611行预测完成！
[ 461.]
第612行预测完成！
[ 357.]
第613行预测完成！
[ 335.]
第614行预测完成！
[ 380.]
第615行预测完成！
[ 371.]
第616行预测完成！
[ 575.]
第617行预测完成！
[ 493.]
第618行预测完成！
[ 365.]
第619行预测完成！
[ 279.]
第620行预测完成！
[ 232.]
第621行预测完成！
[ 166.]
第622行预测完成！
[ 52.]
第623行预测完成！
[ 8.]
第624行预测完成！
[ 1.]
第625行预测完成！
[ 1.]
第626行预测完成！
[ 4.]
第627行预测完成！
[ 8.]
第628行预测完成！
[ 15.]
第629行预测完成！
[ 41.]
第630行预测完成！
[ 156.]
第631行预测完成！
[ 412.]
第632行预测完成！
[ 596.]
第633行预测完成！
[ 501.]
第634行预测完成！
[ 647.]
第635行预测完成！
[ 673.]
第636行预测完成！
[ 512.]
第637行预测完成！
[ 484.]
第638行预测完成！
[ 607.]
第639行预测完成！
[ 558.]
第640行预测完成！
[ 952.]
第641行预测完成！
[ 680.]
第642行预测完成！
[ 461.]
第643行预测完成！
[ 357.]
第644行预测完成！
[ 299.]
第6

In [44]:
nov_data.sort_values(by=['time_stamp','loc_id'],inplace=True)

In [47]:
# 预测的负值处理
nov_data.loc[nov_data.num_of_people<0] = 0

In [45]:
nov_data.iloc[:,:3].to_csv('../datasets/nov_prediction.csv',index=False)

## 4. 线上记录
* 3-9月 前一个非null值填充 去掉二月数据 异常值处理 训练xgb1000轮 线下rmse：377 线上rmse：355
* 3-10月 前一个非null值填充 去掉二月数据 异常值处理 训练xgb2000轮 线上rmse：237
* 2-10月 0填充 增加小时特征 训练xgb2000轮 线上rmse：241
* 4-10月 前一个非null值填充 异常值处理 去掉十一 增加小时特征 训练xgb2000轮 线上rmse：

# 公共函数

In [6]:
# 按地点分组
def locID_split(data):
    res = []
    df = data.copy()
    for i in df.loc_id.unique():
        tmp = df[df.loc_id == i]
        tmp.reset_index(inplace=True)
        tmp.drop('index',inplace=True,axis=1)
        res.append(tmp)
    return res

In [14]:
# 实时预测填充
def real_time_predict(model,data):
    df = data.copy()
    test_list = locID_split(df)
    res = pd.DataFrame()
    for tmp in test_list:
        for i in range(tmp.shape[0]):
            X = xgb.DMatrix(tmp.iloc[i:i+1,3:])
            pred = np.round(model.predict(X,ntree_limit = model.best_iteration))
            print(pred)
            tmp.loc[i,'num_of_people'] = pred
            for j in range(len(columns_24h)):
                tmp.loc[i+j+1,columns_24h[j]] = pred
            for k in range(len(columns_30d)):
                tmp.loc[i+(k+1)*24,columns_30d[k]] = pred
            print('第%s行预测完成！'%i)
        print('******删减前地点数据大小：%s******'%tmp.shape[0])
        tmp.dropna(axis=0,inplace=True)
        print('******删减后地点数据大小：%s******'%tmp.shape[0])
        res = res.append(tmp)
    return res

In [8]:
reg = re.compile('\s+|\t|\n') # 正则匹配空格、Tab键、换行
#列名，历史24小时人数
col_24h='''history_1h_population,history_2h_population,history_3h_population,history_4h_population
    ,history_5h_population,history_6h_population,history_7h_population,history_8h_population
    ,history_9h_population,history_10h_population,history_11h_population,history_12h_population
    ,history_13h_population,history_14h_population,history_15h_population,history_16h_population,history_17h_population
    ,history_18h_population,history_19h_population,history_20h_population,history_21h_population,history_22h_population
    ,history_23h_population'''
columns_24h=re.sub(reg,'',col_24h).split(',')  
#列名，同一时刻历史15天人数
col_30d='''history_1d_population,history_2d_population,history_3d_population,
    history_4d_population,history_5d_population,history_6d_population,
    history_7d_population,history_8d_population,history_9d_population,history_10d_population,history_11d_population,
    history_12d_population,history_13d_population,history_14d_population,history_15d_population,history_16d_population
    ,history_17d_population,history_18d_population,history_19d_population,history_20d_population,history_21d_population
    ,history_22d_population,history_23d_population,history_24d_population,history_25d_population,history_26d_population
    ,history_27d_population,history_28d_population,history_29d_population,history_30d_population'''
columns_30d=re.sub(reg,'',col_30d).split(',')  

In [46]:
def xgboost_model_train(train_X,train_y,test_X,test_y):
    # 模型训练
    xgb_train  = xgb.DMatrix(train_X, train_y)
    xgb_test   = xgb.DMatrix(test_X, test_y)
    params_xgb = {
#         'booster': 'gbtree',                # 取决于使用哪种booster,有两中模型可以选择gbtree和gblinear。gbtree使用基于树的模型进行提升计算，gblinear使用线性模型进行提升计算
        'objective': 'reg:linear',     # 定义学习任务及相应的学习目标，可选的目标函数如下：“reg:linear” —— 线性回归,“reg:logistic”—— 逻辑回归。
        #'multi:softmax'
        'gamma': 0.03,                      # 最小损失减少，模型在默认情况下，对于一个节点的划分只有在其loss function 得到结果大于0的情况下才进行，而gamma 给定了所需的最低loss function的值
        'max_depth': 8,                     # 数的最大深度。缺省值为6，树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合
        'lambda': 4,                        # L2 正则的惩罚系数
        'subsample': 0.8,                   # 用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的从整个样本集合中随机的抽取出50%的子样本建立树模型，这能够防止过拟合。
        'colsample_bytree': 0.8,            # 在建立树时对特征采样的比例。缺省值为1。
#         'eval_metric':'mape_ln',
        'min_child_weight': 8.5,            # 孩子节点中最小的样本权重和，在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。
        'silent': 1,                        # 取0时表示打印出运行时信息，取1时表示以缄默方式运行，不打印运行时信息。缺省值为0。
        'eta': 0.03,                        # 为了防止过拟合，更新过程中用到的收缩步长。通常最后设置eta为0.01~0.2
        'seed': 1000}                       # 随机数的种子。缺省值为0。
    num_boost_round = 2000
    plst  = params_xgb.items()
    model_xgb = xgb.train(plst, xgb_train, num_boost_round,evals=[(xgb_train,'train'),(xgb_test,'test')],early_stopping_rounds=20,verbose_eval=True)
    return model_xgb